In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()  

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA

In [0]:
data_dir = '/dbfs/mnt/images/FoodRecipe'
TRAIN = 'images'
TEST = 'testing'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Out[49]: device(type='cuda', index=0)

In [0]:
transform_train = transforms.Compose([transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 
transform_val = transforms.Compose([transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

In [0]:
Train = datasets.ImageFolder(os.path.join(data_dir, TRAIN), transform = transform_train)
Test = datasets.ImageFolder(os.path.join(data_dir, TEST), transform = transform_val)

In [0]:
print("Loaded {} images under {}".format("Train", len(Train.imgs)))
print("Loaded {} images under {}".format("Train", len(Test.imgs)))
print("{} number of classes".format(len(Train.classes)))

Loaded Train images under 75750
Loaded Train images under 25250
101 number of classes

In [0]:
Train_Loader = torch.utils.data.DataLoader(Train, batch_size = 64, shuffle = True, num_workers = 4)
Test_Loader = torch.utils.data.DataLoader(Test, batch_size = 64, shuffle = False, num_workers = 4)

In [0]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(Train_Loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))

In [0]:
vgg16 = models.vgg16(pretrained = True)
for param in vgg16.features.parameters():
    param.require_grad = True


In [0]:
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend(nn.Sequential(nn.Linear(num_features,4096),nn.ReLU(inplace=True),nn.Linear(4096,4096),nn.ReLU(inplace=True),
               nn.Linear(num_features, 101)).to(device)) 
vgg16.classifier = nn.Sequential(*features) 

In [0]:
vgg16

Out[57]: VGG(
 (features): Sequential(
 (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (1): ReLU(inplace=True)
 (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (3): ReLU(inplace=True)
 (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (6): ReLU(inplace=True)
 (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (8): ReLU(inplace=True)
 (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (11): ReLU(inplace=True)
 (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (13): ReLU(inplace=True)
 (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (15): ReLU(inplace=True)
 (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (18): ReLU(inplace=True)
 (19): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (20): ReLU(inplace=True)
 (21): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (22): ReLU(inplace=True)
 (23): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 (24): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (25): ReLU(inplace=True)
 (26): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (27): ReLU(inplace=True)
 (28): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 (29): ReLU(inplace=True)
 (30): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 )
 (avgpool): AdaptiveAvgPool2d(output_size=(7, 7))
 (classifier): Sequential(
 (0): Linear(in_features=25088, out_features=4096, bias=True)
 (1): ReLU(inplace=True)
 (2): Dropout(p=0.5, inplace=False)
 (3): Linear(in_features=4096, out_features=4096, bias=True)
 (4): ReLU(inplace=True)
 (5): Dropout(p=0.5, inplace=False)
 (6): Linear(in_features=4096, out_features=4096, bias=True)
 (7): ReLU(inplace=True)
 (8): Linear(in_features=4096, out_features=4096, bias=True)
 (9): ReLU(inplace=True)
 (10): Linear(in_features=4096, out_features=101, bias=True)
 )
)

In [0]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target):
    """Computes the precision@k for the specified values of k"""
    batch_size = target.shape[0]

    _, pred = torch.max(output, dim=-1)

    correct = pred.eq(target).sum() * 1.0

    acc = correct / batch_size

    return acc

def train(epoch, data_loader, model, optimizer, criterion):

    iter_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()
    model.train()
    for idx, (data, target) in enumerate(data_loader):
        start = time.time()
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()

        #############################################################################
        # TODO: Complete the body of training loop                                  #
        #       1. forward data batch to the model                                  #
        #       2. Compute batch loss                                               #
        #       3. Compute gradients and update model parameters                    #
        #############################################################################
    
        out = model.forward(data)
        loss = criterion(out, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #############################################################################
        #                              END OF YOUR CODE                             #
        #############################################################################

        batch_acc = accuracy(out, target)

        losses.update(loss, out.shape[0])
        acc.update(batch_acc, out.shape[0])

        iter_time.update(time.time() - start)
        if idx % 10 == 0:
            print(('Epoch: [{0}][{1}/{2}]\t'
                   'Time {iter_time.val:.3f} ({iter_time.avg:.3f})\t'
                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                   'Prec @1 {top1.val:.4f} ({top1.avg:.4f})\t')
                   .format(epoch, idx, len(data_loader), iter_time=iter_time, loss=losses, top1=acc))
    return acc.avg, losses.avg


def validate(epoch, val_loader, model, criterion):
    iter_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()

    num_class = 101
    cm =torch.zeros(num_class, num_class)
    # evaluation loop
    for idx, (data, target) in enumerate(val_loader):
        start = time.time()
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        #############################################################################
        # TODO: Complete the body of training loop                                  #
        #       HINT: torch.no_grad()                                               #
        #############################################################################
        with torch.no_grad():
            out = model(data)
            loss = criterion(out, target)
        #############################################################################
        #                              END OF YOUR CODE                             #
        #############################################################################

        batch_acc = accuracy(out, target)

        # update confusion matrix
        _, preds = torch.max(out, 1)
        for t, p in zip(target.view(-1), preds.view(-1)):
            cm[t.long(), p.long()] += 1

        losses.update(loss, out.shape[0])
        acc.update(batch_acc, out.shape[0])

        iter_time.update(time.time() - start)
        if idx % 10 == 0:
            print(('Epoch: [{0}][{1}/{2}]\t'
               'Time {iter_time.val:.3f} ({iter_time.avg:.3f})\t')
               .format(epoch, idx, len(val_loader), iter_time=iter_time, loss=losses, top1=acc))
    cm = cm / cm.sum(1)
    per_cls_acc = cm.diag().detach().numpy().tolist()
    for i, acc_i in enumerate(per_cls_acc):
        print("Accuracy of Class {}: {:.4f}".format(i, acc_i))

    print("* Prec @1: {top1.avg:.4f}".format(top1=acc))
    return acc.avg, losses.avg, cm
  
def main(train_loader, test_loader, model, criterion, optimizer, epochs, scheduler):
    
    if torch.cuda.is_available():
        model = model.cuda()

    best = 0.0
    best_cm = None
    best_model = None
    train_acc = []
    train_loss = []
    val_acc = []
    val_loss = []
    for epoch in range(epochs):
        # train loop
        train_ac, train_ls = train(epoch, train_loader, model, optimizer, criterion)

        # validation loop
        validation_acc, validation_loss, cm = validate(epoch, test_loader, model, criterion)

        if validation_acc > best:
            best = validation_acc
            best_cm = cm
        train_acc.append(train_ac)
        train_loss.append(train_ls)
        val_acc.append(validation_acc)
        val_loss.append(validation_loss)
        scheduler.step()
    print('Best Prec @1 Acccuracy: {:.4f}'.format(best))
    per_cls_acc = best_cm.diag().detach().numpy().tolist()
    for i, acc_i in enumerate(per_cls_acc):
        print("Accuracy of Class {}: {:.4f}".format(i, acc_i))
    return train_acc, train_loss, val_acc, val_loss

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), momentum= 0.9, lr=0.02)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [0]:
train_acc, train_loss, val_acc, val_loss = main(Train_Loader, Test_Loader, vgg16, criterion, optimizer, 10, exp_lr_scheduler)

Epoch: [0][0/1184]	Time 0.035 (0.035)	Loss 4.6255 (4.6255)	Prec @1 0.0312 (0.0312)	
Epoch: [0][10/1184]	Time 0.855 (0.702)	Loss 4.6506 (4.6169)	Prec @1 0.0000 (0.0156)	
Epoch: [0][20/1184]	Time 0.861 (0.737)	Loss 4.5632 (4.6097)	Prec @1 0.0156 (0.0149)	
Epoch: [0][30/1184]	Time 0.867 (0.750)	Loss 4.5064 (4.5805)	Prec @1 0.0312 (0.0202)	
Epoch: [0][40/1184]	Time 0.868 (0.758)	Loss 4.4831 (4.5591)	Prec @1 0.0625 (0.0267)	
Epoch: [0][50/1184]	Time 0.874 (0.763)	Loss 4.4054 (4.5419)	Prec @1 0.0312 (0.0270)	
Epoch: [0][60/1184]	Time 0.879 (0.768)	Loss 4.2485 (4.5141)	Prec @1 0.0625 (0.0295)	
Epoch: [0][70/1184]	Time 0.884 (0.771)	Loss 4.2441 (4.4631)	Prec @1 0.0938 (0.0363)	
Epoch: [0][80/1184]	Time 0.887 (0.775)	Loss 3.9830 (4.4208)	Prec @1 0.0469 (0.0411)	
Epoch: [0][90/1184]	Time 0.891 (0.777)	Loss 3.8274 (4.3870)	Prec @1 0.1250 (0.0452)	
Epoch: [0][100/1184]	Time 0.901 (0.783)	Loss 4.0999 (4.3502)	Prec @1 0.0625 (0.0476)	
Epoch: [0][110/1184]	Time 0.888 (0.784)	Loss 3.9097 (4.3165)	Prec @1 0.0625 (0.0511)	
Epoch: [0][120/1184]	Time 0.888 (0.786)	Loss 4.0645 (4.2923)	Prec @1 0.0625 (0.0540)	
Epoch: [0][130/1184]	Time 0.901 (0.788)	Loss 4.1812 (4.2641)	Prec @1 0.0312 (0.0568)	
Epoch: [0][140/1184]	Time 0.905 (0.789)	Loss 3.4618 (4.2329)	Prec @1 0.1719 (0.0609)	
Epoch: [0][150/1184]	Time 0.901 (0.791)	Loss 3.7848 (4.2008)	Prec @1 0.1094 (0.0662)	
Epoch: [0][160/1184]	Time 0.907 (0.792)	Loss 3.9122 (4.1703)	Prec @1 0.0781 (0.0689)	
Epoch: [0][170/1184]	Time 0.910 (0.794)	Loss 3.5266 (4.1367)	Prec @1 0.1719 (0.0736)	
Epoch: [0][180/1184]	Time 0.909 (0.795)	Loss 3.9242 (4.1080)	Prec @1 0.0625 (0.0780)	
Epoch: [0][190/1184]	Time 0.906 (0.796)	Loss 3.7460 (4.0838)	Prec @1 0.1719 (0.0813)	
Epoch: [0][200/1184]	Time 0.906 (0.797)	Loss 3.5774 (4.0570)	Prec @1 0.1094 (0.0855)	
Epoch: [0][210/1184]	Time 0.906 (0.798)	Loss 2.9064 (4.0320)	Prec @1 0.3438 (0.0889)	
Epoch: [0][220/1184]	Time 0.907 (0.799)	Loss 3.6764 (4.0166)	Prec @1 0.0312 (0.0914)	
Epoch: [0][230/1184]	Time 0.910 (0.800)	Loss 3.9623 (3.9995)	Prec @1 0.1094 (0.0947)	
Epoch: [0][240/1184]	Time 0.911 (0.801)	Loss 3.7852 (3.9888)	Prec @1 0.1562 (0.0969)	
Epoch: [0][250/1184]	Time 0.911 (0.802)	Loss 3.2099 (3.9666)	Prec @1 0.2031 (0.1005)	
Epoch: [0][260/1184]	Time 0.908 (0.802)	Loss 3.4049 (3.9494)	Prec @1 0.1406 (0.1028)	
Epoch: [0][270/1184]	Time 0.905 (0.803)	Loss 3.1229 (3.9286)	Prec @1 0.2500 (0.1064)	
Epoch: [0][280/1184]	Time 0.906 (0.803)	Loss 3.5374 (3.9086)	Prec @1 0.1250 (0.1090)	
Epoch: [0][290/1184]	Time 0.919 (0.804)	Loss 3.6359 (3.8962)	Prec @1 0.1719 (0.1122)	
Epoch: [0][300/1184]	Time 0.913 (0.805)	Loss 3.7685 (3.8838)	Prec @1 0.1094 (0.1140)	
Epoch: [0][310/1184]	Time 0.911 (0.805)	Loss 3.5900 (3.8724)	Prec @1 0.1250 (0.1156)	
Epoch: [0][320/1184]	Time 0.911 (0.806)	Loss 2.9778 (3.8543)	Prec @1 0.2344 (0.1183)	
Epoch: [0][330/1184]	Time 0.921 (0.806)	Loss 3.6571 (3.8420)	Prec @1 0.1250 (0.1205)	
Epoch: [0][340/1184]	Time 0.908 (0.807)	Loss 3.4196 (3.8282)	Prec @1 0.1875 (0.1226)	
Epoch: [0][350/1184]	Time 0.909 (0.807)	Loss 3.0921 (3.8126)	Prec @1 0.2500 (0.1249)	
Epoch: [0][360/1184]	Time 0.919 (0.807)	Loss 2.8594 (3.7953)	Prec @1 0.2656 (0.1280)	
Epoch: [0][370/1184]	Time 0.908 (0.808)	Loss 3.5253 (3.7814)	Prec @1 0.1875 (0.1301)	
Epoch: [0][380/1184]	Time 0.910 (0.808)	Loss 3.1436 (3.7658)	Prec @1 0.2969 (0.1331)	
Epoch: [0][390/1184]	Time 0.911 (0.809)	Loss 3.2317 (3.7502)	Prec @1 0.1875 (0.1362)	
Epoch: [0][400/1184]	Time 0.909 (0.809)	Loss 3.6256 (3.7383)	Prec @1 0.1094 (0.1382)	
Epoch: [0][410/1184]	Time 0.911 (0.809)	Loss 2.9443 (3.7258)	Prec @1 0.2969 (0.1406)	
Epoch: [0][420/1184]	Time 0.914 (0.810)	Loss 3.0577 (3.7144)	Prec @1 0.2812 (0.1428)	
Epoch: [0][430/1184]	Time 0.912 (0.810)	Loss 2.7955 (3.7014)	Prec @1 0.2969 (0.1448)	
Epoch: [0][440/1184]	Time 0.914 (0.810)	Loss 3.2757 (3.6885)	Prec @1 0.2656 (0.1472)	
Epoch: [0][450/1184]	Time 0.911 (0.811)	Loss 2.7152 (3.6733)	Prec @1 0.2656 (0.1500)	
Epoch: [0][460/1184]	Time 0.909 (0.811)	Loss 2.8739 (3.

In [0]:
import matplotlib.pyplot as plt
def my_plot(epochs, loss):
    plt.plot(epochs, loss)
my_plot(np.linspace(1, 10, 10).astype(int), train_acc)
my_plot(np.linspace(1, 10, 10).astype(int), val_acc)
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.savefig('/dbfs/mnt/images/accuracy_0.02_true.png')

In [0]:
my_plot(np.linspace(1, 10, 10).astype(int), train_loss)
my_plot(np.linspace(1, 10, 10).astype(int), val_loss)
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.savefig('/dbfs/mnt/images/loss_0.02_true.png')